In [1]:
# Colab

# !pip install -U rapidsai
# import cudf


In [2]:
!git clone https://github.com/kkonstantin182/wine-classifier.git
%cd wine-classifier

Cloning into 'wine-classifier'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 89 (delta 43), reused 49 (delta 14), pack-reused 0
Unpacking objects: 100% (89/89), 1.31 MiB | 3.68 MiB/s, done.
/content/wine-classifier


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: scikit_learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [4]:
# !pip install pandarallel
!python -m spacy download it_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16690 sha256=60543cb5da007b6c85e6f25237d79f8fe4336cfd75bd9055cf96cf2315e7b1d1
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel
2023-05-07 12:59:50.349554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 12:59:51.437490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, 

In [5]:
import sys
from notebooks.path_conf import get_project_root
path_src = get_project_root() / "src"
sys.path.append(str(path_src.resolve()))

from src.text_processing import TextProcessing
from src.dataset import Dataset
from src.constants import SEED


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
from bornrule import BornClassifier
import multiprocessing as mp
from tqdm import tqdm
# from pandarallel import pandarallel

In [7]:
# Initializtion of parall-n
# pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [8]:
data_path = get_project_root() / "data" / "dataset1_proc.csv"

In [9]:
TARGET_MAP = {
    'Rosato': 0, 
    'Frizzante': 1, 
    'Bianco': 2, 
    'Rosso': 3
}

COLUMNS = {
    'target': ['type'],
    'text': ['review', 'winery', 'variety'],
    'numerical': ['price'],
    'categorical': ['appellation2']
}

In [10]:
ds_obj = Dataset(data_path, target_map=TARGET_MAP, columns_names=COLUMNS)

<bound method Dataset.print_shape of <src.dataset.Dataset object at 0x7ff85a4e7610>>


In [11]:
train_set, test_set = ds_obj()

The train shape: (12000, 4)
The test shape: (3000, 4)


In [12]:
tp_obj_clean = TextProcessing(is_lemmatized=True)

In [ ]:
# with mp.Pool(4) as pool:
#     train_set['text'] = (pool.imap(tp_obj_clean, train_set['text']), total=len(train_set)))

# with mp.Pool(4) as pool:
#     test_set['text'] = list(tqdm(pool.imap(tp_obj_clean, test_set['text']), total=len(test_set)))

In [14]:
with mp.Pool(mp.cpu_count()) as pool:
    test_set['text'] = pool.map(tp_obj_clean, test_set['text'])

In [18]:
test_set.head(2)

,target,price,appellation2,text
6968,3,47.0,Piemonte,Delicati aromio fiore blu bacca rosso accenno ...
5190,2,31.0,Veneto,bianco radioso emergere delicato sentore fiore...


In [16]:
with mp.Pool(mp.cpu_count()) as pool:
    train_set['text'] = pool.map(tp_obj_clean, train_set['text'])

In [17]:
train_set.head(2)

,target,price,appellation2,text
14109,3,15.0,Centro Italia,realizzato 85 Sangiovese 15 Cabernet Sauvignon...
6673,3,20.0,Toscana,carne griglia fuoco legna contrastano amae re ...


In [19]:
train_set.to_csv('dataset1_train_text_cleaned.csv')
test_set.to_csv('dataset1_test_text_cleaned.csv')